In [4]:
!nvidia-smi

Thu Aug  8 19:31:12 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.76                 Driver Version: 560.76         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1050      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   61C    P0             N/A / ERR!  |     104MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import cv2
import os
import albumentations as A
from PIL import Image
import numpy as np

In [2]:
# Fungsi untuk menerapkan CLAHE
def apply_clahe(image_path, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(img)
    cv2.imwrite(output_path, cl1)

# Penerapan CLAHE pada semua gambar
folders = ['train/images_ori', 'valid/images_ori', 'test/images_ori']
output_folders = ['train/images_clahe', 'valid/images', 'test/images']

for input_folder, output_folder in zip(folders, output_folders):
    os.makedirs(output_folder, exist_ok=True)
    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        output_path = os.path.join(output_folder, img_name)
        apply_clahe(img_path, output_path)

In [7]:
def augment_image(image):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.GaussianBlur(blur_limit=(3, 5), p=0.5),
        A.ElasticTransform(alpha=1, sigma=50, alpha_affine=None, p=0.5),
        A.GridDistortion(p=0.5),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5)
    ])
    augmented_image = transform(image=image)['image']
    return augmented_image

# penerapan augmentasi pada semua gambar di folder train
input_folder = 'train/images_clahe'
output_folder = 'train/images'
os.makedirs(output_folder, exist_ok=True)

for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)
    image = np.array(Image.open(img_path))
    augmented_image = augment_image(image)
    augmented_image = Image.fromarray(augmented_image)
    augmented_image.save(os.path.join(output_folder, img_name))


In [8]:
from ultralytics import YOLO

In [11]:
# Load model YOLOv8m
model = YOLO('yolov8n.pt')

In [15]:
# Melatih model
results = model.train(data='data.yaml', epochs=50, imgsz=512, batch=16, device='cuda:0')

Ultralytics YOLOv8.2.74  Python-3.12.0 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning C:\Users\ASUS\Documents\myProject\pneumonia\train\labels... 4810 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4810/4810 [00:07<00:00, 637.14it/s] 


train: New cache created: C:\Users\ASUS\Documents\myProject\pneumonia\train\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\ASUS\Documents\myProject\pneumonia\valid\labels... 901 images, 0 backgrounds, 0 corrupt: 100%|██████████| 901/901 [00:07<00:00, 115.53it/s]


val: New cache created: C:\Users\ASUS\Documents\myProject\pneumonia\valid\labels.cache
Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        901       1425      0.495      0.366      0.379       0.14

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50       1.5G      2.146      2.718      2.207         15        512: 100%|██████████| 301/301 [02:23<00:00,  2.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:12<00:00,  2.25it/s]


                   all        901       1425      0.495      0.374       0.38       0.14

50 epochs completed in 2.376 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.2.74  Python-3.12.0 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:12<00:00,  2.28it/s]


                   all        901       1425      0.533      0.359      0.381      0.143
Speed: 0.2ms preprocess, 6.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\detect\train


In [16]:
# Menyimpan model yang telah dilatih
model.save('trained_model/trained_model.pth')

In [17]:
# Menampilkan metrik pelatihan
print(results)

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000018CF77058E0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480